# Import libraries

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re

C:\Users\Nandan Hegde\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\Nandan Hegde\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Load datasets

In [2]:
tmdb_movies = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\tmdb_5000_movies.csv')
tmdb_credits = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\tmdb_5000_credits.csv')
netflix_titles = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\netflix_titles.csv')
movies = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\movies.csv')
ratings = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\ratings.csv')
tags = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\tags.csv')
links = pd.read_csv(r'C:\Users\Nandan Hegde\OneDrive\Documents\GitHub\MSU_CMSE_830_Final_Semester_project\Movie_analysis_dashboard\Data_sources\links.csv')

# Data Preprocessing

In [3]:
# Preprocessing Titles: Function to clean and standardize titles
def clean_title(title):
    title = re.sub(r"[^a-zA-Z0-9\s]", "", title)  # Remove special characters
    title = re.sub(r"\s+", " ", title).strip()  # Normalize whitespace
    return title.lower()

# Apply title cleaning
netflix_titles['cleaned_title'] = netflix_titles['title'].apply(clean_title)
tmdb_movies['cleaned_title'] = tmdb_movies['title'].apply(clean_title)

# Step 1: Combine TMDB Movies and TMDB Credits
tmdb_combined = tmdb_movies.merge(tmdb_credits, left_on='id', right_on='movie_id', how='left')

# Step 2: Aggregate MovieLens Ratings by movieId
movielens_ratings_agg = ratings.groupby('movieId').agg(
    avg_rating=('rating', 'mean'),
    rating_count=('rating', 'count')
).reset_index()

# Step 3: Merge MovieLens Movies with aggregated Ratings data
movielens_combined = movies.merge(movielens_ratings_agg, on='movieId', how='left')

# Step 4: Aggregate MovieLens Tags by movieId
movielens_tags_agg = tags.groupby('movieId').agg(
    tags=('tag', lambda x: ', '.join(x))
).reset_index()

# Add tags to the MovieLens Combined data
movielens_combined = movielens_combined.merge(movielens_tags_agg, on='movieId', how='left')

# Step 5: Merge TMDB and MovieLens using 'links' (movieId -> tmdbId)
movielens_with_tmdb = movielens_combined.merge(links, on='movieId', how='left').merge(
    tmdb_combined, left_on='tmdbId', right_on='id', how='left'
)

# Extract release year from TMDB for joining with Netflix
movielens_with_tmdb['release_year'] = pd.to_datetime(movielens_with_tmdb['release_date'], errors='coerce').dt.year
netflix_titles['release_year'] = pd.to_datetime(netflix_titles['release_year'], errors='coerce').dt.year

# Fuzzy Matching: Adjusted threshold with preprocessing and additional filtering
netflix_title_map = {}
tmdb_titles = movielens_with_tmdb[['title', 'release_year', 'cleaned_title']].dropna()

for index, row in netflix_titles.iterrows():
    netflix_title = row['cleaned_title']
    netflix_year = row['release_year']
    possible_matches = tmdb_titles[tmdb_titles['release_year'] == netflix_year]
    
    # Get best fuzzy match using a lower threshold and exclude poor matches
    best_match = process.extractOne(netflix_title, possible_matches['cleaned_title'], scorer=fuzz.token_set_ratio)
    if best_match and best_match[1] > 70:  # Lowering threshold to 70
        best_match_title = best_match[0]
        netflix_title_map[(best_match_title, netflix_year)] = index

# Integrating Netflix data based on the improved fuzzy matching
netflix_columns = ['show_id', 'type', 'director', 'cast', 'country', 'date_added', 'rating', 'duration', 'listed_in', 'description']
for col in netflix_columns:
    movielens_with_tmdb[f'netflix_{col}'] = None

for index, row in movielens_with_tmdb.iterrows():
    title = row['cleaned_title']
    year = row['release_year']
    
    if (title, year) in netflix_title_map:
        netflix_index = netflix_title_map[(title, year)]
        for col in netflix_columns:
            movielens_with_tmdb.at[index, f'netflix_{col}'] = netflix_titles.at[netflix_index, col]

# Final Combined Dataset
combined_analytical_data = movielens_with_tmdb

# Display the structure and confirm Netflix data integration
combined_analytical_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               9742 non-null   int64  
 1   title                 9742 non-null   object 
 2   genres_x              9742 non-null   object 
 3   avg_rating            9724 non-null   float64
 4   rating_count          9724 non-null   float64
 5   tags                  1572 non-null   object 
 6   imdbId                9742 non-null   int64  
 7   tmdbId                9734 non-null   float64
 8   budget                3537 non-null   float64
 9   genres_y              3537 non-null   object 
 10  homepage              1301 non-null   object 
 11  id                    3537 non-null   float64
 12  keywords              3537 non-null   object 
 13  original_language     3537 non-null   object 
 14  original_title        3537 non-null   object 
 15  overview             

In [4]:
combined_analytical_data.head()

,movieId,title,genres_x,avg_rating,rating_count,tags,imdbId,tmdbId,budget,genres_y,...,netflix_show_id,netflix_type,netflix_director,netflix_cast,netflix_country,netflix_date_added,netflix_rating,netflix_duration,netflix_listed_in,netflix_description
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,215.0,"pixar, pixar, fun",114709,862.0,30000000.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""...",...,None,None,None,None,None,None,None,None,None,None
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,110.0,"fantasy, magic board game, Robin Williams, game",113497,8844.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,52.0,"moldy, old",113228,15602.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,7.0,NaN,114885,31357.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,5,Father of the Bride Part II (1995),Comedy,3.071429,49.0,"pregnancy, remake",113041,11862.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [5]:
combined_analytical_data.netflix_description.unique()

array([None,
       "A soldier grows disillusioned as he's caught between rival sergeants in this semi-autobiographical Vietnam War drama from writer-director Oliver Stone.",
       "When Earth's superheroes exile the volatile Incredible Hulk to a distant planet called Sakaar, the muscle-bound green monster is sold into slavery.",
       'While gathering evidence against his employer to help the FBI build a price-fixing case, Mark Whitacre begins to piece together a fantasy world.',
       "Ex-con Toro's brother and former partner in crime makes a desperate plea for help that may put Toro back behind bars ... if he survives at all.",
       "A man becomes the prime suspect in his wife's murder, but the police investigation soon widens to other potential culprits with hidden motives.",
       'A youthful drug dealer in Mumbai must change his ways when the sister who raised him is assaulted by her boss and imprisoned for defending herself.'],
      dtype=object)